# DataTracer Quickstart

In this short tutorial we will guide you through the necessary steps
to get started using **DataTracer**.

In the following sections we will:

* Generate and load some demo data
* Get the list of available pipelines
* Detect the primary keys of a dataset
* Detect the foreign keys of a dataset

## Load some data

The first step will be to get and load some data to work with.

For this, we will use the function `get_demo_data`, which will
generate a folder in our current working directory with some
datasets inside it.

In [1]:
from datatracer import get_demo_data

get_demo_data(force=True)

Generating a demo folder at `./datatracer_demo`


Afterwards, we can load all the generated datasets using the `load_datasets` function.

In [2]:
from datatracer import load_datasets

datasets = load_datasets('datatracer_demo')

This will return us a dict with tuples containing a MetaData instance with information
about the dataset and a dictionary with the tables loaded as `pandas.DataFrame` instances.

In [3]:
datasets.keys()

dict_keys(['mutagenesis', 'Chess', 'classicmodels', 'university', 'Bupa', 'trains', 'SameGen', 'NBA', 'pubs'])

In [4]:
metadata, tables = datasets.pop('classicmodels')

In [5]:
{
    table['name']: table['primary_key']
    for table in metadata.get_tables()
}

{'customers': 'customerNumber',
 'employees': 'employeeNumber',
 'offices': 'officeCode',
 'orderdetails': ['orderNumber', 'productCode'],
 'orders': 'orderNumber',
 'payments': ['customerNumber', 'checkNumber'],
 'productlines': 'productLine',
 'products': 'productCode'}

In [6]:
metadata.get_table_names()

['customers',
 'employees',
 'offices',
 'orderdetails',
 'orders',
 'payments',
 'productlines',
 'products']

In [7]:
list(tables.keys())

['customers',
 'employees',
 'offices',
 'orderdetails',
 'orders',
 'payments',
 'productlines',
 'products']

In [8]:
metadata.get_table('customers')

{'name': 'customers',
 'primary_key': 'customerNumber',
 'fields': [{'name': 'customerNumber', 'data_type': 'numerical'},
  {'name': 'customerName', 'data_type': 'text'},
  {'name': 'contactLastName', 'data_type': 'text'},
  {'name': 'contactFirstName', 'data_type': 'text'},
  {'name': 'phone', 'data_type': 'text'},
  {'name': 'addressLine1', 'data_type': 'text'},
  {'name': 'addressLine2', 'data_type': 'text'},
  {'name': 'city', 'data_type': 'text'},
  {'name': 'state', 'data_type': 'text'},
  {'name': 'postalCode', 'data_type': 'text'},
  {'name': 'country', 'data_type': 'text'},
  {'name': 'salesRepEmployeeNumber', 'data_type': 'numerical'},
  {'name': 'creditLimit', 'data_type': 'numerical'}]}

In [9]:
tables['customers'].head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",NaN,Nantes,NaN,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,NaN,Las Vegas,NV,83030,USA,1166.0,71800.0
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300.0
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200.0
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,NaN,Stavern,NaN,4110,Norway,1504.0,81700.0


For the rest of this tutorial, we will try to solve a data lineage problem using
the dataset that we just explored, using the rest of the datasets as our training data.

In [10]:
training_datasets = list(datasets.values())

## Select a Pipeline

In the DataTracer project, the Data Lineage problems are solved using what we call _pipelines_.

Each _pipeline_ solves a different type of Data Lineage problem:

* **Primary Key detection**: Identify which column is the primary key in each table.
* **Foreign Key detection**: Find which relationships exist between the tables.
* **Column Mapping**: Given a field in a table, deduce which other fields, from the same table or other tables, are more related to or contributed the most in generating the given field.

We can see the list of available pipelines using the `get_pipelines` function:

In [11]:
from datatracer import get_pipelines

get_pipelines()

['datatracer.column_map.basic',
 'datatracer.foreign_key.basic',
 'datatracer.foreign_key.standard',
 'datatracer.primary_key.basic']

In the next steps we will use the `datatracer.primary_key.basic` pipeline to detect the
primary keys of our dataset, and later on use the `datatracer.foreign_key.standard` to
obtain a list of foreing key candidates.

## Use a DataTracer instance to find primary keys

In order to use a pipeline you will need to create a `DataTracer` instance passing the name of
the pipeline that we want to use.

In [12]:
from datatracer import DataTracer

dtr = DataTracer('datatracer.primary_key.basic')

Once we have our instance we will need to fit it to our training data to make it learn
from other datasets.

In [13]:
dtr.fit(training_datasets)

And once it has been fitted we are ready to solve our Data Lineage problem by calling
the `solve` method passing the tables that we want to analyze.

In [14]:
primary_keys = dtr.solve(tables)

This will return us a dictionary containing the name of the primary key of each table.

In [15]:
primary_keys

{'customers': 'customerNumber',
 'employees': 'employeeNumber',
 'offices': 'officeCode',
 'orderdetails': 'orderNumber',
 'orders': 'orderNumber',
 'payments': 'customerNumber',
 'productlines': 'productLine',
 'products': 'productCode'}

## Use the DataTracer to find foreign keys

Now let's try to solve a more complex problem, such as as figuring out the relationships between our tables.

For this, we will use the `datatracer.foreign_key.standard` pipeline.

In [16]:
from datatracer import DataTracer

dtr = DataTracer('datatracer.foreign_key.standard')
dtr.fit(training_datasets)
foreign_keys = dtr.solve(tables)

extracting features: 100%|██████████| 8/8 [00:00<00:00, 19.07it/s]


The output will be a list containing all the foreign candidates that the DataTracer considered
valid.

In [17]:
foreign_keys

[{'table': 'products',
  'field': 'productLine',
  'ref_table': 'productlines',
  'ref_field': 'productLine'},
 {'table': 'payments',
  'field': 'customerNumber',
  'ref_table': 'customers',
  'ref_field': 'customerNumber'},
 {'table': 'orders',
  'field': 'customerNumber',
  'ref_table': 'customers',
  'ref_field': 'customerNumber'},
 {'table': 'orderdetails',
  'field': 'productCode',
  'ref_table': 'products',
  'ref_field': 'productCode'},
 {'table': 'orderdetails',
  'field': 'orderNumber',
  'ref_table': 'orders',
  'ref_field': 'orderNumber'},
 {'table': 'employees',
  'field': 'officeCode',
  'ref_table': 'offices',
  'ref_field': 'officeCode'}]

## Solve a Column Mapping problem

Now that we are able to figure out the relationships between our tables, we will try to go one
step further and try to solve the Data Lineage problem of knowing, for a given column of our
tables, which other columns were most likely involved in the computation of its values.

For this, we will use the `datatracer.column_map` pipeline passing two additional arguments to
the `solve` method:

* `target_table`: The name of the table which contains the column that we want to analyze.
* `target_field`: The name fo the column that we want to analyze.

For example, in this case we will try to see which fields are more closely related with
the field `quantityOrdered` from the table `orderdetails`.

In [18]:
from datatracer import DataTracer

dtr = DataTracer('datatracer.column_map.basic')
dtr.fit(training_datasets)
column_map = dtr.solve(
    tables,
    target_table='orderdetails',
    target_field='quantityOrdered'
)

extracting features: 100%|██████████| 8/8 [00:00<00:00, 18.00it/s]


The output will be a dict containing the fields that DataTracer consider that are more closely
related to the value of the given field, with a score indicating how strong the relationship is.

In [19]:
column_map

{('orderdetails', 'orderNumber'): 0.3854540808399761,
 ('orderdetails', 'priceEach'): 0.43313416486749556,
 ('orderdetails', 'orderLineNumber'): 0.18141175429252837}

For example, in this case we can see that the field that has the strongest relationship with the
`quantityOrdered` field is the `priceEach` of the same table.

Now, just for demonstration purposes, let's make an experiment and create a new field in the same table which is directly derived from the `quantityOrdered` field. For example, let's multiply it by 2:

In [20]:
tables['orderdetails']['quantityOrdered_x2'] = tables['orderdetails']['quantityOrdered'] * 2

Now we can use DataTracer again to see if it is able to detect this new field as a candidate:

In [21]:
column_map = dtr.solve(
    tables,
    target_table='orderdetails',
    target_field='quantityOrdered'
)

In [22]:
column_map

{('orderdetails', 'orderNumber'): 0.00022471076974350297,
 ('orderdetails', 'priceEach'): 0.00015177917546761553,
 ('orderdetails', 'orderLineNumber'): 0.00014925873321498974,
 ('orderdetails', 'quantityOrdered_x2'): 0.9994742513215739}

As we can see, DataTracer has successfully detected the new field and selected it as
the one with the strongest relationship with our target field.